# Implementation of "Fair Decisions Despite Imperfect Predictions" using the original fairness constraint

In [1]:
import numpy as np
from src.consequential_learning import collect_data, train
from src.feature_map import IdentityFeatureMap

## The parameters used by the the original authors  
Note: Learning rate decay is not yet implemented

In [2]:
training_parameters = {
    'dim_x': 1,
    'dim_s': 1,
    'time_steps':200,
    'batch_size':512,
    'num_iterations': 32,
    'learning_parameters': {
        'learning_rate': 0.5,
        'decay_rate': 0.8,
        'decay_step': 30
    },
    'fairness_rate':0,
    'cost_factor':0.55,
    'fraction_protected':0.3
}
training_parameters['dim_theta'] = training_parameters['dim_x'] + training_parameters['dim_s']
training_parameters['feature_map'] = IdentityFeatureMap(training_parameters['dim_theta'])
training_parameters['num_decisions'] = training_parameters['num_iterations'] * training_parameters['batch_size']

## Definition of the fairness function  
The original fairness constraint was defined as the difference of benefits $b_{P}^s$ for both of the settings of the protected attribute. This function defines both the fairness function as well as its derivative which is controlled by the parameter gradient=true/false

In [3]:
def fairness_function(**fairness_kwargs):
    return fairness_kwargs['policy'].benefit_difference(fairness_kwargs['x'], fairness_kwargs['s'], fairness_kwargs['sample_theta'], fairness_kwargs['gradient'])


## Train the model

In [4]:
train(**training_parameters, fairness_function=fairness_function)

Time step 1: Utility -0.019372625903688295
Time step 2: Utility 0.04927891156462583
Time step 3: Utility 0.054648788456302735
Time step 4: Utility 0.06197144045473449
Time step 5: Utility 0.061748223072717316
Time step 6: Utility 0.06884374147804743
Time step 7: Utility 0.06793041308566673
Time step 8: Utility 0.06266014602562335
Time step 9: Utility 0.06577716643741402
Time step 10: Utility 0.0687388744351636
Time step 11: Utility 0.06683369447453953
Time step 12: Utility 0.06325417644622393
Time step 13: Utility 0.06640157694399128
Time step 14: Utility 0.06703527815468113
Time step 15: Utility 0.0660620008122377
Time step 16: Utility 0.06240494296577945
Time step 17: Utility 0.06880797452582027
Time step 18: Utility 0.07320207820617992
Time step 19: Utility 0.06855418418554182
Time step 20: Utility 0.07699008308764406
Time step 21: Utility 0.06570264765784112
Time step 22: Utility 0.06882201436898466
Time step 23: Utility 0.07089078129224112
Time step 24: Utility 0.062510148849797
T